<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2022_GoogleColabs/blob/main/Week5/MatrixMultiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#prepare setup of cuda plugin for jupyter notebook
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

/usr/local
  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 3145740     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-02-04 04:43:58.278274999 +0000
Modify: 2022-02-04 04:43:58.167275204 +0000
Change: 2022-02-04 04:43:58.167275204 +0000
 Birth: -
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-v2u4cety
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-v2u4cety
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=1a5ac0771015358159482d68ab1075ac0e1506cdbf935bab62957f0153ca6bc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-8gwtkj_t/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
directory /usr/local/src already exists
Out bin /usr/local/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>

__global__ void MatrixMulKernel(float* d_M, float* d_N, float* d_P, int Width) {
	// Calculate the row index of the d_Pelement and d_M
	int Row = blockIdx.y*blockDim.y+threadIdx.y;
	// Calculate the column index of d_P and d_N
	int Col = blockIdx.x*blockDim.x+threadIdx.x;
	if ((Row < Width) && (Col < Width)) {
		float Pvalue = 0;
		// each thread computes one element of the block sub-matrix
		for (int k = 0; k < Width; ++k) {
			Pvalue += d_M[Row*Width+k]*d_N[k*Width+Col];
		}
		d_P[Row*Width+Col] = Pvalue;
	}
}

void MatrixMultHost(float* A, float* B, float* C, int N)
{
	for (int i = 0; i < N; ++i)
	{
		for (int j = 0; j < N; ++j)
		{
			float Pvalue = 0;
			for (int k = 0; k < N; ++k)
			{
				Pvalue += A[i*N + k] * B[k*N + j];
			}
			C[j + i*N] = Pvalue;
		}
	}
}

int main(int argc, char** argv)
{
	const int N = 4;
	const int SIZE = N*N;
	const int SIZE_IN_BYTES = SIZE * sizeof(float);

	float* h_A = (float*)malloc(SIZE_IN_BYTES);
	float* h_B = (float*)malloc(SIZE_IN_BYTES);
	float* h_C = (float*)malloc(SIZE_IN_BYTES);
	float* h_CD = (float*)malloc(SIZE_IN_BYTES); //device calc res

	// Initialize matrices on the host
	for (int i = 0; i < N; i++) {
		 for (int j = 0; j < N; j++) {
			 h_A[i*N + j] = (float) (rand() % 1024);
			 h_B[i*N + j] = (float) (rand() % 1024);
		 }
	}

	float* d_A;
	float* d_B;
	float* d_C;

	cudaMalloc(&d_A, SIZE_IN_BYTES);
	cudaMalloc(&d_B, SIZE_IN_BYTES);
	cudaMalloc(&d_C, SIZE_IN_BYTES);

	cudaMemcpy(d_A, h_A, SIZE_IN_BYTES, cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, h_B, SIZE_IN_BYTES, cudaMemcpyHostToDevice);

	dim3    blocksGrid;
	dim3    threadsBlock(16, 16, 1);

	blocksGrid.x = (N + threadsBlock.x - 1) / threadsBlock.x;
	blocksGrid.y = (N + threadsBlock.y - 1) / threadsBlock.y;

	float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

	// some events to count the execution time
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	// start to count execution time of GPU version
	cudaEventRecord(start, 0);

	MatrixMulKernel << <blocksGrid, threadsBlock >> > (d_A, d_B, d_C, N);

	cudaMemcpy(h_CD, d_C, SIZE_IN_BYTES, cudaMemcpyDeviceToHost);
	 
	// time counting terminate
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);

	// compute time elapse on GPU computing
	cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
	printf("Time elapsed (GPU): %f ms.\n", gpu_elapsed_time_ms);

	// start the CPU version
	cudaEventRecord(start, 0);
	MatrixMultHost(h_A, h_B, h_C, N);

	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
	printf("Time elapsed (CPU): %f ms.\n", cpu_elapsed_time_ms);

	//validate results 
	// validate results computed by GPU
	int all_ok = 1;
	for (int i = 0; i < N; ++i)
	{
		for (int j = 0; j < N; ++j)
		{ 
			if (h_C[j*N + i] != h_CD[j*N + i])
			{
				all_ok = 0;
			}
		} 
	}

	// roughly compute speedup
	if (all_ok)
	{
		printf("All results are correct!!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
	}
	else
	{
		printf("incorrect results\n");
	}


	free(h_A);
	free(h_B);
	free(h_C);
	free(h_CD);

	cudaFree(&d_A);
	cudaFree(&d_B);
	cudaFree(&d_C);

	cudaDeviceReset();
	return 0;
}

Time elapsed (GPU): 0.189152 ms.
Time elapsed (CPU): 0.003264 ms.
All results are correct!!!, speedup = 0.017256

